In [ ]:
import pandas, numpy, seaborn, sklearn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

## functions & options


In [ ]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# I] Dataframe settings

In [ ]:
%%time
#Open the big expression data with all the transcripts in 3 parts. Chunksize is number of rows.
path = output_file_directory + "/dataframes_for_input/all_transcripts_zscore.tsv"
df = pandas.read_csv(path, sep = "\t", index_col='sample')

print(df.shape)
df.head()

In [1]:
row_10p = int(df.shape[0]/10)
df_small = df.iloc[:row_10p,:]

NameError: name 'df' is not defined

# IV] Result: Are they in the same expression cluster ? 

In [ ]:
%%time
# 9 is the best number of cluster
kmeans_model = KMeans(n_clusters = 4, random_state=1, verbose=False).fit(df_small)
labels = kmeans_model.labels_
# Find what cluster for each gene
pred_cluster = kmeans_model.predict(df_small)
print(pred_cluster)
#add the prediction in a column
df_small['cluster'] = pred_cluster
df_small.head()

In [ ]:
#Final answer
ATG7_1 = 'ENST00000354449.7'
ATG7_2 = 'ENST00000354956.9'

# print(df['cluster'][[ATG7_1, ATG7_2]])

In [ ]:
# print('The two transcripts are not in the same cluster. BINGO ')

# IV] Clustermap

In [ ]:
seaborn.hls_palette(4)

In [ ]:
# %%time
# # #pop the cluster column
# cluster_uniq = df_small.pop("cluster")
# #use one color per cluster group
# lut = dict(zip(cluster_uniq.unique(), seaborn.hls_palette(4)))
# row_colors_clust = cluster_uniq.map(lut)
# #plot
# seaborn.clustermap(df_small, cmap='bwr', col_cluster=True, row_cluster=True,
#                    row_colors= row_colors_clust, 
#                    method = "complete", metric = 'correlation',
#                    yticklabels= '', xticklabels = '',
#                    cbar_kws={'label':'expression [z-score]'})

In [ ]:
df_small

In [ ]:
#sort the df_small with the cluster numbers.
df_small = df_small.sort_values('cluster')

#save index of 1st row of each clusters for horizontal line
nb_cluster = 4
horiz_line_pos = []
print('First row for each clusters:')
for i in range(1,nb_cluster):
    
    first_index = df_small.reset_index().loc[df_small.reset_index()['cluster'].isin([i])].index[0]
    horiz_line_pos.append(first_index)
print(horiz_line_pos)

# calculate the number of transcripts per cluster
fst_index_clust = horiz_line_pos
length = len(fst_index_clust)
print(f'\nNumber of transcripts per cluster:')
print(f'cluster 1 = {fst_index_clust[0]}')
for i in range(1, length):
    print(f'cluster {i+1} = {fst_index_clust[i] - fst_index_clust[i-1]}')
print(f'cluster {i+2} = {len(df_small) - fst_index_clust[i]}')

In [ ]:
%%time
# #pop the cluster column
cluster_uniq = df_small.pop("cluster")

#use one color per cluster group 
lut = dict(zip(cluster_uniq.unique(), seaborn.hls_palette(4)))
row_colors_clust = cluster_uniq.map(lut)

#plot
seaborn.clustermap(df_small, cmap='bwr', col_cluster=True, row_cluster=False,
                   center=0, vmin=-12, vmax=12,
                   row_colors= row_colors_clust, 
                   yticklabels= '', xticklabels = '',
                   cbar_kws={'label':'expression [log2(tpm+1)]'})

plt.hlines(horiz_line_pos, xmin=0, xmax=4386, colors = 'black', label='LAAA')
plt.show()
plt.close()

## Why don't see the hlines ??? 

In [ ]:
print('Why plt.hlines() is not working ?')

In [ ]:
x = [1, 2, 3]
y = numpy.array([[1, 2], [3, 4], [5, 6]])
plt.plot(x, y)

adada = [5,4,3,2]
plt.hlines(adada, xmin=0, xmax=3, colors = 'black')

In [ ]:
print("I thnik it works but I don't see hlines because they are to tiny")

In [ ]:
#ATG7 (1) and (2) are in cluster 2 and 0 respectively

In [ ]:
%%time
#reinstall the cluster column
df['cluster'] = cluster_uniq
print( set(df['cluster']) )

cluster_atg7_1 = df.loc[df.index == ATG7_1]['cluster'][0]
cluster_atg7_2 = df.loc[df.index == ATG7_2]['cluster'][0]

#create df with the entire cluster where iso1 and iso2 are
df_2_clusters =  df[ (df['cluster'] == cluster_atg7_1) | (df['cluster'] == cluster_atg7_2) ]
print( set(df_2_clusters['cluster']) )

#pop the cluster column
cluster_uniq = df_2_clusters.pop("cluster")

#use one color per cluster group 
lut = dict(zip(cluster_uniq.unique(), seaborn.hls_palette(2)))
row_colors_clust = cluster_uniq.map(lut)

#plot
seaborn.clustermap(df_2_clusters, cmap='bwr', col_cluster=True, row_cluster=False,
                   center=0, 
#                    vmin=-8, vmax=8,
                   row_colors= row_colors_clust, 
                   yticklabels= '', xticklabels = '',
                   cbar_kws={'label':'expression [log2(tpm+1)]'})

plt.show()
plt.close()

In [ ]:
%%time
#all select ATG7(1) and (2)
ATG7_1 = 'ENST00000354449.7'
ATG7_2 = 'ENST00000354956.9'
ensembl_atg7_list = [ATG7_1, ATG7_2]

#create df with only ATG7(1) and (2) from df_clust1_2
df_iso1_2 =  df_2_clusters[df_2_clusters.index.isin(ensembl_atg7_list)]
print(df_iso1_2.shape)

#use one color per isoform 
lut = dict(zip(ensembl_atg7_list, seaborn.hls_palette(2)))
row_colors_isof = df_iso1_2.index.map(lut)

#plot
seaborn.clustermap(df_iso1_2, cmap='bwr', col_cluster=True, row_cluster=False,
                   center=0, 
#                    vmin=-8, vmax=8,
                   row_colors= row_colors_clust, 
                   yticklabels= '', xticklabels = '',
                   cbar_kws={'label':'expression [log2(tpm+1)]'})

plt.show()
plt.close()